In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.data import Dataset

In [2]:
df_sub = pd.read_csv(Path.cwd() / 'data' / 'sample_submission.csv')

In [3]:
df_train = pd.read_csv(Path.cwd() / 'data' / 'train.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_test = pd.read_csv(Path.cwd() / 'data' / 'test.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df_train.drop_duplicates(inplace=True)

In [6]:
dataset = Dataset()

In [7]:
train_data, test_data = train_test_split(df_train, random_state=32, shuffle=True)

In [12]:
train_data.userid.max()

127495

In [8]:
dataset.fit(train_data.userid.unique(), train_data.itemid.unique())

In [9]:
dataset.model_dimensions()

(126552, 41137)

In [28]:
(interactions, weights) = dataset.build_interactions(
    [(x[11], x[12], x[13]) for x in train_data.values])

In [30]:
model = LightFM(loss='warp')

In [31]:
model.fit(interactions, # spase matrix representing whether user u and item i interacted
      sample_weight= weights, # spase matrix representing how much value to give to user u and item i inetraction: i.e ratings
      epochs=10)

In [33]:
from lightfm.evaluation import auc_score
train_auc = auc_score(model,
                      interactions
                     ).mean()
print('Hybrid training set AUC: %s' % train_auc)

Hybrid training set AUC: 0.90070546


In [39]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [40]:
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
NUM_EPOCHS = 25 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [13]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)

NameError: name 'model' is not defined

In [42]:
roc_auc_score(test_data.rating,preds)

0.7244333171669821

In [44]:
preds_sub = model.predict(df_test.userid.values,
                      df_test.itemid.values)

In [45]:
len(preds_sub)

285965

In [49]:
preds.min(), preds.max()

(-35.288516998291016, 38.3861083984375)

In [46]:
normalized_preds = (preds_sub - preds_sub.min())/(preds_sub - preds_sub.min()).max()

In [47]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [48]:
df_sub['rating']= normalized_preds

In [49]:
df_sub.to_csv('submission_log.csv', index=False)